### Imports

In [42]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule
import hdnntools as hdt

import numpy as np
import ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "2"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units
from ase.optimize import LBFGS
from ase.io import read, write
from ase.parallel import world
from ase import Atom, Atoms

import matplotlib.pyplot as plt

### Setup files to load (2LUF or TrpCage)

In [43]:
molfile = 'l.pdb'

ntdir = '/home/jsmith48/ANI-Networks/networks/ANI-1x_wcoulomb-3/'
cns = ntdir + 'rHCNO-5.0R_16-3.4A_a8-4.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 4

### Load ANI models

In [44]:
# Setup NeuroChem
aens = ensemblemolecule(cns, sae, nnf, Nn, 0)

In [45]:
# Load molecule
mol = read(molfile)

# Set ANI calculator
mol.set_calculator(ANIENS(aens))

### Calculate energy and force

In [46]:
s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

print('Energy:',E)
print('Energy:',F)

spc = mol.get_chemical_symbols()
print('Number of atoms:',len(spc))

Total time: 0.30655956268310547
Energy: -779747.1524971446
Energy: [[ 1.152414    0.44286013  0.3826418 ]
 [-1.1272864  -0.32571375  0.7263681 ]
 [ 0.19105546 -0.46433198 -0.8786344 ]
 ...
 [ 0.02854292  0.3865103   1.037487  ]
 [ 0.42189258 -0.8220571  -0.84911555]
 [-0.5597946   0.32895553  0.02199788]]
Number of atoms: 1125


mol.

In [47]:
X = mol.get_positions()
Q = mol.calc.nc.compute_mean_charges()[0]

Q.size/3

375.0

In [48]:
D = []
for i in range(375):
    D.append(np.linalg.norm(np.sum((Q[3*i:3*i+3]*X[3*i:3*i+3].T).T,axis=0))/0.20819434)

In [49]:
np.mean(D)

2.9183236923927325

### Run MD

### Visualization